system requirements
1. activate gedi env, cuda 11.7
2. install tinycudann

In [16]:
import torch
import numpy as np
import open3d as o3d
import torch.nn.functional as F
from gedi import GeDi
import torch.nn as nn
import math
import os

from models.fields import FeatureField
from pyhocon import ConfigFactory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('You are using the following device: ', device)

"""script for extract DINO feature from rendered image and geometric features from pointcloud, visualizing feature matching results
"""
def sample_point_cloud_from_mesh(mesh_path, number_of_points=100000,save_to_file=True):
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    
    # Check if the mesh has vertex normals, if not, compute them
    if not mesh.has_vertex_normals():
        mesh.compute_vertex_normals()
    
    # Sample points uniformly from the mesh
    pcd = mesh.sample_points_uniformly(number_of_points=number_of_points)

    if save_to_file:
        pcd_fname = mesh_path.replace("glb","pcd")
        if not os.path.exists(pcd_fname):
            print(f'cache extracted pcd file to {pcd_fname}')
            o3d.io.write_point_cloud(pcd_fname, pcd)
    
    return pcd

def load_pointcloud(pcd_fname):
    if os.path.exists(pcd_fname):
        print(f"read cached pcd file from {pcd_fname}")
        pcd = o3d.io.read_point_cloud(pcd_fname)
    else:
        print("can't find cached pcd file, read from mesh file")
        mesh_path = pcd_fname.replace("pcd","glb")
        pcd = sample_point_cloud_from_mesh(mesh_path)
    return pcd

def visualize_point_cloud(pcd):
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])


def load_checkpoint(checkpoint_fname):
    # read network config
    conf_path = './confs/wmask.conf'
    f = open(conf_path)
    conf_text = f.read()
    conf_text = conf_text.replace('CASE_NAME', 'owl') #TODO case name as input
    f.close()
    conf = ConfigFactory.parse_string(conf_text)

    # load feature field
    checkpoint = torch.load(os.path.join(checkpoint_fname), map_location='cuda')
    feature_network = FeatureField(**conf['model.feature_field']).to('cuda')
    # self.nerf_outside.load_state_dict(checkpoint['nerf'])
    # self.sdf_network.load_state_dict(checkpoint['sdf_network_fine'])
    # self.deviation_network.load_state_dict(checkpoint['variance_network_fine'])
    # self.color_network.load_state_dict(checkpoint['color_network_fine'])
    feature_network.load_state_dict(checkpoint['feature_network'])
    # self.optimizer_geometry.load_state_dict(checkpoint['optimizer-geometry'])
    # self.optimizer_feature.load_state_dict(checkpoint['optimizer-feature'])

    # iter_step = checkpoint['iter_step']
    print(f"loaded checkpoint from {checkpoint_fname}")
    return feature_network


def visualize_point_cloud(pcd):
    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

You are using the following device:  cuda


In [53]:
def extract_geo_feature(pcd=None, config=None):
    if config == None:
        config = {
          'dim': 32,                                            # descriptor output dimension
          'samples_per_batch': 500,                             # batches to process the data on GPU
          'samples_per_patch_lrf': 4000,                        # num. of point to process with LRF
          'samples_per_patch_out': 512,                         # num. of points to sample for pointnet++
          'r_lrf': .5,                                          # LRF radius
          'fchkpt_gedi_net': 'data/chkpts/3dmatch/chkpt.tar',   # path to checkpoint
          'device':'cpu',
        }  
    voxel_size = .01
    patches_per_pair = 5000

    # initialising class
    gedi = GeDi(config=config)

    # getting a pair of point clouds
    if pcd==None: # if not passing pcd directly, read from local storage
        pcd0 = o3d.io.read_point_cloud('data/assets/threed_match_7-scenes-redkitchen_cloud_bin_0.ply')
    else:
        pcd0 = pcd

    pcd0.paint_uniform_color([1, 0.706, 0])

    # estimating normals (only for visualisation)
    pcd0.estimate_normals()

    # randomly sampling some points from the point cloud
    inds0 = np.random.choice(np.asarray(pcd0.points).shape[0], patches_per_pair, replace=False) # 5000

    pts0 = torch.tensor(np.asarray(pcd0.points)[inds0]).float() # 5000 x 3 

    # applying voxelisation to the point cloud
    pcd0 = pcd0.voxel_down_sample(voxel_size)

    # pcd in tensor
    _pcd0 = torch.tensor(np.asarray(pcd0.points)).float()

    # computing descriptors
    pcd0_desc = gedi.compute(pts=pts0, pcd=_pcd0) # 5000x32

    return pcd0_desc, pts0

# is scale kept consist from MESH to pcd then goes back? are geometry feature point and dino feature point the same point?
def extract_DINO_feature_from_model(model_ckpt, pts):
    model = load_checkpoint(model_ckpt)
    return model(pts)



In [54]:
pcd_path_source = "./exp/neus/mixer/meshes/mixer_5000.pcd" 
pcd_source= load_pointcloud(pcd_path_source)
geo_features, sampled_pts = extract_geo_feature(pcd_source) # tensor on cpu, bc some operation dosen't support cuda

# move tensor to GPU 
geo_features = geo_features.to(device)
sampled_pts = sampled_pts.to(device)

# extract DINO feature
ckpt = './exp/neus/mixer/checkpoints/ckpt_005000.pth'
DINO_features = extract_DINO_feature_from_model(ckpt,sampled_pts)

# combine features
features = torch.cat((geo_features,DINO_features),1)
# features = DINO_features

# pick one point and expand features
target_indx = np.random.randint(0,len(sampled_pts)) 
target_pt = sampled_pts[target_indx]
target_feature = features[target_indx].reshape(1,-1)

read cached pcd file from ./exp/neus/mixer/meshes/mixer_5000.pcd
loaded checkpoint from ./exp/neus/mixer/checkpoints/ckpt_005000.pth


In [60]:
# extract features from second pointcloud
pcd_path_source2 = "./exp/neus/owl/meshes/owl.pcd" 
pcd_source2= load_pointcloud(pcd_path_source2)
geo_features2, sampled_pts2 = extract_geo_feature(pcd_source2) # tensor on cpu, bc some operation dosen't support cuda

# move tensor to GPU 
geo_features2 = geo_features2.to(device)
sampled_pts2 = sampled_pts2.to(device)

# extract DINO feature
ckpt = './exp/neus/owl/checkpoints/ckpt_010000.pth'
DINO_features2 = extract_DINO_feature_from_model(ckpt,sampled_pts2)

# combine features
features2 = torch.cat((geo_features2,DINO_features2),1)
# features2 = DINO_features2

read cached pcd file from ./exp/neus/owl/meshes/owl.pcd
loaded checkpoint from ./exp/neus/owl/checkpoints/ckpt_010000.pth


### feature matching


In [61]:
features = target_feature.repeat(features2.shape[0],1)
# Normalize the features to get cosine similarity
features1_norm = F.normalize(features, p=2, dim=1)
features2_norm = F.normalize(features2, p=2, dim=1)


# Compute cosine similarity between all pairs
cosine_sim = torch.mm(features1_norm, features2_norm.t())

# Flatten the cosine similarity matrix and get the top 10 indices
num_rank = 10
cosine_sim_flat = cosine_sim.view(-1)
topk_values, topk_indices = torch.topk(cosine_sim_flat, num_rank, largest=True)

# Manually convert flat indices to 2D indices
n_rows = cosine_sim.size(0)
topk_indices_2d = [(idx // n_rows, idx % n_rows) for idx in topk_indices]

print("Top 10 closest pairs (indices and cosine similarities):")
for i in range(num_rank):
    idx1, idx2 = topk_indices_2d[i]
    print(f"Pair {i+1}: (features1[{idx1}], features2[{idx2}]), Cosine Similarity: {topk_values[i].item()}")

Top 10 closest pairs (indices and cosine similarities):
Pair 1: (features1[7], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 2: (features1[6], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 3: (features1[4], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 4: (features1[5], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 5: (features1[1], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 6: (features1[0], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 7: (features1[2], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 8: (features1[3], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 9: (features1[8], features2[64]), Cosine Similarity: 0.2330734133720398
Pair 10: (features1[9], features2[64]), Cosine Similarity: 0.2330734133720398


### visualization

In [62]:
# Extract the corresponding points for the top pairs
top_pairs_pts1 = np.array([target_pt.cpu().numpy() for idx1, _ in topk_indices_2d])
top_pairs_pts2 = np.array([sampled_pts2[idx2].cpu().numpy()+1.0 for _, idx2 in topk_indices_2d])

# Create a new point cloud to hold the combined points for visualization
combined_points = np.vstack((sampled_pts.cpu(), sampled_pts2.cpu()+1.0))
combined_pcd = o3d.geometry.PointCloud()
combined_pcd.points = o3d.utility.Vector3dVector(combined_points)

# Create lines connecting the top pairs
# lines = [[idx1.cpu().numpy(), (idx2 + len(sampled_pts)).cpu().numpy()] for idx1, idx2 in topk_indices_2d]
lines = [[i, i + len(top_pairs_pts1)] for i in range(len(top_pairs_pts1))]

# Visualize point clouds with lines between top pairs
colors = [[0, 0, 1] for _ in range(len(lines))]  # Blue lines

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(np.vstack((top_pairs_pts1, top_pairs_pts2)))
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point clouds and the lines
o3d.visualization.draw_geometries([combined_pcd, line_set])

[[-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]
 [-0.08489986  0.31720576  0.36402828]]
[[0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]
 [0.6743127 1.018662  1.1245183]]
[[0, 10], [1, 11], [2, 12], [3, 13], [4, 14], [5, 15], [6, 16], [7, 17], [8, 18], [9, 19]]


-----------------
### past results


In [ ]:
# 5000 to 5000 code
# Normalize the features to get cosine similarity
features1_norm = F.normalize(features, p=2, dim=1)
features2_norm = F.normalize(features2, p=2, dim=1)

# Compute cosine similarity between all pairs
cosine_sim = torch.mm(features1_norm, features2_norm.t())

# Flatten the cosine similarity matrix and get the top 10 indices
num_rank = 3
cosine_sim_flat = cosine_sim.view(-1)
topk_values, topk_indices = torch.topk(cosine_sim_flat, num_rank, largest=True)

# Manually convert flat indices to 2D indices
n_rows = cosine_sim.size(0)
topk_indices_2d = [(idx // n_rows, idx % n_rows) for idx in topk_indices]

print("Top 10 closest pairs (indices and cosine similarities):")
for i in range(num_rank):
    idx1, idx2 = topk_indices_2d[i]
    print(f"Pair {i+1}: (features1[{idx1}], features2[{idx2}]), Cosine Similarity: {topk_values[i].item()}")